In [ ]:
!pip install transformers accelerate
!pip install datasets
!pip install transformers[sentencepiece]
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from transformers import BertModel, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer
from sklearn.model_selection import train_test_split
import evaluate
import torch
import random
import numpy as np

#The dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# (3 points) Fine-tune the model on the training data



In [5]:
# Define a pretrained model and tokenizer. 
# We use distilBERT, a smaller version of BERT that retains most of its performance but is much faster.
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Preprocessing the data using a tokenizer
def tokenize(batch):
    """
    Tokenize the input text using a pretrained tokenizer.

    Args:
    batch: A batch of text data.

    Returns:
    A dictionary containing the tokenized text and any other required model inputs.
    """
    return tokenizer(batch['text'], padding=True, truncation=True)

# Prepare datasets
train_dataset = train_df.map(tokenize, batched=True, batch_size=len(train_df))
valid_dataset = valid_df.map(tokenize, batched=True, batch_size=len(valid_df))

# Format the datasets to output torch tensors
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
valid_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Basics args
training_args = TrainingArguments(
    output_dir="./test-tain",
    num_train_epochs=1,
)

# Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

trainer.train()

trainer.evaluate()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.418100
1000,0.329000
1500,0.294500
2000,0.260800
2500,0.260100


{'eval_loss': 0.2609541416168213,
 'eval_runtime': 77.8894,
 'eval_samples_per_second': 64.194,
 'eval_steps_per_second': 8.024,
 'epoch': 1.0}

# (2 points) Evaluate the model in term of accuracy on the test data.

In [6]:
test_dataset = test_df.map(tokenize, batched=True, batch_size=len(test_df))
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
predictions = trainer.predict(test_dataset)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [7]:
print(predictions[0].shape)
print(predictions[1].shape)

logits = torch.from_numpy(predictions[0])
labels = torch.from_numpy(predictions[1])

# Convertir les logits en prédictions de classe
predictions = torch.argmax(logits, dim=1)

# Comparer les prédictions aux labels pour obtenir le nombre total de prédictions correctes
correct_predictions = torch.sum(predictions == labels)

# Calculer l'accuracy
accuracy = correct_predictions.float() / labels.size(0)

print(f"Accuracy: {accuracy.item()}")

(25000, 2)
(25000,)
Accuracy: 0.9240400195121765


The model has an accuracy of 92% on the test data which is very good.

# (2 points) For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [9]:
misclassified_indexes = (predictions != labels).nonzero().numpy().flatten()

for index in misclassified_indexes[:2]:
    print(f'Text: {test_df["text"][index]}')
    print(f'Expected: {labels[index].item()}')
    print(f'Got: {predictions[index].item()}\n')

Text: First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
Expected: 0
Got: 1

Text: Isaac Florentine has made some of the best western Martial Arts action movies ever produced. In particular US Seals 2, Cold Harvest, Special Forces and Undisputed 2 are all action classics. You can tell Isaac has a real passion for the genre and his films are always eventful, creative and sharp

+ The first text is relatively complex as it is labeled as negative, yet the author describes a rather positive film that he enjoyed but which will not appeal to everyone.
+ The second text is also complex as the author describes many positive things but in the end, he says he did not like it. It's a matter of context.

# (3 points) What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

When comparing models for production, we can consider the following:

- Naive Bayes:

 + Advantages: Simple, interpretable, works well with small datasets.
 + Disadvantages: Limited performance but it's also because it's simple.

- Transformer-based Models (like this one):

 + Advantages: High performance, capture complex patterns and context in text data.
 + Disadvantages: Resource-intensive, as we can just training for one epoch is very long.

- RNNs/LSTMs:

  + Advantages: Handle long sequences, capture temporal dependencies.
  + Disadvantages: Limited scalability for very long sequences, slower compared to transformer-based models.

In summary, Naive Bayes is simple and interpretable but has limited performance. Transformer-based models offer high performance but require more resources and are less interpretable. RNNs/LSTMs handle long sequences but are slower. So to choose one it depends on what we want to do.